<a href="https://colab.research.google.com/github/mauricioyc/Probabilistic-Machine-Learning/blob/master/Markov_Chain_Monte_Carlo/Markov_Chain_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>